In [449]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import pickle
from more_itertools import unique_everseen

import faiss
import pandas as pd
import numpy as np

from sem_covid.entrypoints.notebooks.legal_radar.services.split_documents_pipeline import DOCUMENT_ID_SOURCE
from sem_covid.services.store_registry import store_registry
from sem_covid.services.model_registry import embedding_registry
from sem_covid import config
from sem_covid.services.sc_wrangling.feature_selector import reduce_array_column

In [454]:
filter_year = [1900, 2021, (1900, 2021), 1]

In [3]:
FAISS_BUCKET_NAME = 'faiss-index'
FAISS_INDEX_FINREG_NAME = 'faiss_index_finreg.pkl'
FIN_REG_SPLITTED_ES_INDEX = 'ds_finreg_splitted'
DATES_DOCUMENT = 'dates_document'
HTML_LINKS = 'htmls_to_download'
DEFAULT_SEARCH = """The Semantic Interoperability Community develops solutions to help European public administrations perform seamless and meaningful cross-border and cross-domain data exchanges."""
TEXT_PIECE = 'text_piece'

In [473]:
def load_documents():
    """Read the data from ES."""
    es_store = store_registry.es_index_store()
    df = es_store.get_dataframe(index_name=config.EU_FINREG_CELLAR_ELASTIC_SEARCH_INDEX_NAME)
    df[DATES_DOCUMENT] = pd.to_datetime(df[DATES_DOCUMENT]).dt.date
    return df

def load_splitted_documents():
    """Read the data from ES."""
    es_store = store_registry.es_index_store()
    return es_store.get_dataframe(index_name=FIN_REG_SPLITTED_ES_INDEX)

def load_emb_model():
    return embedding_registry.sent2vec_universal_sent_encoding()

def load_faiss_index():
    """Load and deserialize the Faiss index."""
    minio_store = store_registry.minio_object_store(minio_bucket=FAISS_BUCKET_NAME)
    data = pickle.loads(minio_store.get_object(object_name=FAISS_INDEX_FINREG_NAME))
    return faiss.deserialize_index(data)


def semantic_search(user_input: str):
    documents = load_documents()
    splitted_documents = load_splitted_documents()
    model = load_emb_model()
    faiss_index = load_faiss_index()
    num_results = 100
    embeddings = model.encode(sentences=[user_input])
    D, I = faiss_index.search(np.array(embeddings).astype("float32"), k=num_results)
    document_parts = pd.DataFrame(splitted_documents.iloc[I.flatten().tolist()])
    document_parts['similarity'] = pd.Series(D.flatten().tolist()).values
    document_parts = document_parts.drop_duplicates(DOCUMENT_ID_SOURCE).reset_index(drop=True)
    documents_id = document_parts[DOCUMENT_ID_SOURCE].values
    result_documents = pd.DataFrame(documents.loc[documents_id]).reset_index(drop=True)
    result_documents = pd.DataFrame(frame).reset_index(drop=True)
    result_documents['similarity'] = document_parts['similarity']
    result_documents['text_piece'] = document_parts['text_piece']
    
    return result_documents

In [386]:
doc = load_splitted_documents()

100% (69866 of 69866) |##################| Elapsed Time: 0:00:25 Time:  0:00:25


In [390]:
doc['text_piece']

_id
b1b019e1b3e8acaada8bf3a6f1375559f7c055503a998e18929c083874f20845    Council Regulation (EC) No 695/2005 of 26 Apri...
f0a9df6bcc52a8c3141143e6f5c930f1229b0cdd86c410f81ba5ac36382c422c    THE COUNCIL OF THE EUROPEAN UNION, Having rega...
918fae7f7068d4543bd02df2a3cc71074c6e7c00c828c5ad03e2bb00229eadf5    (5) Application of this mechanism should be te...
5100524aeb02b779514c85d9c16996f642350643699b1312bee1ea5cf550d224    It shall apply to expenditure incurred from 1 ...
ab3f9e3bb321a7a03419439e620abc5628effc8928589da662a04967269a90be    Council Regulation (EC) No 3234/94 of 20 Decem...
                                                                                          ...                        
bbfe3907ed0ab3745e63d568a501cc492bb9a6a152e0fb492741f06e559a6f26    OSOBA/ (SL) V JAVNI LISTINI, KI JI JE PRILOZEN...
4c083494d2d65d1bf113d32aba6f69169a8af7c91dfd48375dd803334bd2ece2    FORMULAR AR BIFOGAT NAMNS INGEN UPPGIFT OM DOM...
62bef0c8780712d7d68fa61626b5e9ba2b5e494b579a2e88a8c1

In [385]:
sample_questions['Questions/Text Extracts'][0]

'‘clearing’ means the process of establishing positions, including the calculation of net obligations, and ensuring that financial instruments, cash, or both, are available to secure the exposures arising from those positions;'

In [356]:
# documents, celex number and url reference from sample question file
sample_questions = pd.read_csv('sem_covid/entrypoints/notebooks/legal_radar/docs/sample_questions_v4.csv').dropna()

In [430]:
# def execution_verification(dataset: pd.DataFrame, source_documents_column: str, celex_number_column: str) -> pd.DataFrame:
    # """
    #     This function helps us to compare the celex id from our source and legal radar semantic search.
    #     Inserting source document text and his celex number, checks if each document is found in the first
    #     10 results.
    # Args:
    #     dataset: dataframe with required source document and his celex number
    #     source_documents_column: dataframe's source document column name
    #     celex_number_column: dataframe's celex number column name

    # Returns: the name of inserted document and how much results that are having their celex number.
    # """
#     results = []

#     for (source_document, celex_number) in zip(dataset[source_documents_column], dataset[celex_number_column]):
#         executed_search_engine = main(source_document)
#         reduced_execution = reduce_array_column(executed_search_engine, 'celex_numbers').reset_index(drop=True)
#         sum_celex_number_found = reduced_execution['celex_numbers'].isin([celex_number])
#         index = reduced_execution[sum_celex_number_found].index.to_list()

#         results.append(
#             {
#                 'position_in_documents': reduced_execution['content'].loc[index].index.to_list(),
#                 'position_in_slices': reduced_execution['text_piece'].loc[index].index.to_list(),
#                 'sum_of_values': sum_celex_number_found.sum(),
#                 "similarity_percentage": reduced_execution['similarity'].apply(lambda x: 1 / (1 + x)).loc[index].to_list()
#             }
#         )
#     result = pd.DataFrame(results).applymap(lambda x: x[0] if isinstance(x, list) else x)
    
#     result['in_top_5_slices'] = result['position_in_slices'] <= 5
#     result['in_top_10_slices'] = result['position_in_slices'] <= 10
#     result['in_top_5_documents'] = result['position_in_documents'] <= 5
#     result['in_top_10_documents'] = result['position_in_documents'] <= 10
#     result['in_q3'] = result['similarity_percentage'] >= 0.75

#     return pd.concat([dataset, result], axis=1)


# def execute_semantic_search(dataset: pd.DataFrame, input_query: str, celex_number: str) -> list:
#         results = []
        
#         for (query_text, celex_number) in zip(dataset[input_query], dataset[celex_number]):
#             executed_search_engine = semantic_search(query_text)
#             reduced_execution = reduce_array_column(executed_search_engine, 'celex_numbers').reset_index(drop=True)
#             sum_celex_number_found = reduced_execution['celex_numbers'].isin([celex_number])
#             index = reduced_execution[sum_celex_number_found].index.to_list()

#             results.append({
#                     'position_in_documents': reduced_execution['content'].loc[index].index.to_list(),
#                     'position_in_slices': reduced_execution['text_piece'].loc[index].index.to_list(),
#                     'sum_of_values': sum_celex_number_found.sum(),
#                     "similarity_percentage": reduced_execution['similarity'].apply(lambda x: 1 / (1 + x)).loc[index].to_list()})
            
#         return results
    

# def view_semantic_search_results(semantic_search_results) -> pd.DataFrame:
#         result = pd.DataFrame(semantic_search_results).applymap(lambda x: x[0] if isinstance(x, list) else x)
#         result = result.assign(in_top_5_slices=result['position_in_slices'] <= 5,
#                                in_top_10_slices=result['position_in_slices'] <= 10,
#                                in_top_5_documents=result['position_in_documents'] <= 5,
#                                in_top_10_documents=result['position_in_documents'] <= 10,
#                                in_q3=result['similarity_percentage'] >= 0.75)
        
#         return pd.concat([dataset, result], axis=1)


class SemanticSearchEvaluation:
    """Test evaluation for Legal Radar - Semantic Search"""
    def __init__(self, dataset: pd.DataFrame, input_query: str, celex_number: str) -> None:
        """
            dataset: dataframe with required source document and his celex number
            source_documents_column: dataframe's source document column name
            celex_number_column: dataframe's celex number column name
        """
        self.dataset = dataset
        self.input_query = input_query
        self.celex_number = celex_number
    
    def execute_semantic_search(self) -> list:
        """
            From inserted dataframe, it finds the column with queries and the values 
            and executes the semantic search function. IT calculates the sum of found numbers
            and gets their indexes. After that it appends into a list the possitions of 
            documents and documents' slices, sum of found values and similarity percentage.
            
        """
        results = []
        
        for (query_text, celex_number) in zip(self.dataset[self.input_query], self.dataset[self.celex_number]):
            executed_search_engine = semantic_search(query_text)
            reduced_execution = reduce_array_column(executed_search_engine, 'celex_numbers').reset_index(drop=True)
            sum_celex_number_found = reduced_execution['celex_numbers'].isin([celex_number])
            index = reduced_execution[sum_celex_number_found].index.to_list()

            results.append({
                    'position_in_documents': reduced_execution['content'].loc[index].index.to_list(),
                    'position_in_slices': reduced_execution['text_piece'].loc[index].index.to_list(),
                    'sum_of_values': sum_celex_number_found.sum(),
                    "similarity_percentage": reduced_execution['similarity'].apply(lambda x: 1 / (1 + x)).loc[index].to_list()})
            
        return results

    def view_semantic_search_results(self) -> pd.DataFrame:
        """
            With founded results, it drops the lists from the table and assigns new columns with
            boolean statement of founded celex numbers in first 5, 10 search results of documents and
            documents' slices and checks if the similarity of those results are more than Q3, and concatenates
            the original inserted dataset and new generated one with found data.
        """
        result = pd.DataFrame(self.execute_semantic_search())
        result = result.assign(in_top_5_slices=result['position_in_slices'].apply(lambda x: any(np.array(x) <= 5)),
                               in_top_10_slices=result['position_in_slices'].apply(lambda x: any(np.array(x) <= 10)),
                               in_top_5_documents=result['position_in_documents'].apply(lambda x: any(np.array(x) <= 5)),
                               in_top_10_documents=result['position_in_documents'].apply(lambda x: any(np.array(x) <= 10)),
                               in_q3=result['similarity_percentage'].apply(lambda x: any(np.array(x) >= 0.75)))
        
        return pd.concat([self.dataset, result], axis=1)


In [416]:
data = {'a': [[0, 1], [9, 8]], 'b': [[4, 6], [9, 10]]}
df = pd.DataFrame(data=data)
df

,a,b
0,"[0, 1]","[4, 6]"
1,"[9, 8]","[9, 10]"


In [ ]:
df['a'].astype()

In [426]:
df['result'] = df['a'].apply(lambda x: any(np.array(x) <= 8))
df

,a,b,result
0,"[0, 1]","[4, 6]",True
1,"[9, 8]","[9, 10]",True


In [431]:
execution = SemanticSearchEvaluation(sample_questions.iloc[[0, 1, 2]], 'Questions/Text Extracts', 'Celex No').view_semantic_search_results()

100% (5791 of 5791) |####################| Elapsed Time: 0:00:03 Time:  0:00:03
100% (69866 of 69866) |##################| Elapsed Time: 0:00:26 Time:  0:00:26
100% (5791 of 5791) |####################| Elapsed Time: 0:00:05 Time:  0:00:05
100% (69866 of 69866) |##################| Elapsed Time: 0:00:25 Time:  0:00:25
100% (5791 of 5791) |####################| Elapsed Time: 0:00:05 Time:  0:00:05
100% (69866 of 69866) |##################| Elapsed Time: 0:00:25 Time:  0:00:25


In [433]:
execution

,No,Questions/Text Extracts,Source Document,Article No,Celex No,Work Reference,ELI,position_in_documents,position_in_slices,sum_of_values,similarity_percentage,in_top_5_slices,in_top_10_slices,in_top_5_documents,in_top_10_documents,in_q3
0,1,‘clearing’ means the process of establishing p...,REGULATION (EU) No 648/2012 OF THE EUROPEAN PA...,2(3),32012R0648,http://publications.europa.eu/resource/cellar/...,http://data.europa.eu/eli/reg/2012/648/oj,[27],[27],1,[0.4546839568469639],False,False,False,False,False
1,2,Incentives to promote the use of CCPs have not...,REGULATION (EU) No 648/2012 OF THE EUROPEAN PA...,Recital 13,32012R0648,http://publications.europa.eu/resource/cellar/...,http://data.europa.eu/eli/reg/2012/648/oj,[0],[0],1,[0.641506835303643],True,True,True,True,False
2,3,Ensuring that the clearing obligation reduces ...,REGULATION (EU) No 648/2012 OF THE EUROPEAN PA...,Recital 15,32012R0648,http://publications.europa.eu/resource/cellar/...,http://data.europa.eu/eli/reg/2012/648/oj,[0],[0],1,[0.659670742152862],True,True,True,True,False


In [435]:
sample_questions['Questions/Text Extracts'][0]

'‘clearing’ means the process of establishing positions, including the calculation of net obligations, and ensuring that financial instruments, cash, or both, are available to secure the exposures arising from those positions;'

In [474]:
a = semantic_search(sample_questions['Questions/Text Extracts'][0])

100% (5791 of 5791) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
100% (69866 of 69866) |##################| Elapsed Time: 0:00:25 Time:  0:00:25


document_id: ['cc5be0d28ee5b0332b1ba70264ba503e94a5aebde3b8f6ee5ac10178f05dfb22', '3dd9b6cc01d7ceb47f5addf73d17845808128fdec6816fab4d8694c522b3cf22', '4dd677b38398ec579370d6f632f3dc6b25ba03aaa95f5af671cd03cdf33b414f', '1a75b7c864a130ae577a580f9d9c499bac1541815211eb54089fc653728425e8', '9f35ac2834f1c5359f69bcfd1bfaa278771d8a62046ed6f764d59ac010aa1b7b', '61660b7423a61edefd2f9b705f8e7f258749522640e639715ded991c336389e9', '94bdddc03e01cf0350bebedf6d68851fb00b42e1c08331b6628575c25bc92802', '6d9613bc0f79d4a801cb05e11b36ba3c2903192b49efb1f172ab789586c0fe26', 'a3a3dade855dda6b799759793d6cdad1c7d32719dc9f0c4129ec0535be387018', '43dbae6d59364c70647defcb43657c8da82e46e57a13aabe532ffd7ff60d240a', 'd62e551cad3e60e90ecf350eb5f4a786fd0726a24f6b3886d32400de29bfbd63', '6f766754cebf6526d11d50c4320f6504fbc17bb21127441e930d9dd1282f7c68', 'd5979fec26d25e1ef34623aa09e8e3df0b0c501489ea0817bf5649d2754b7c40', 'a162b15e98539c424d93b31762bae3e5ed37661cbf5ae41dff0e127a117b13e7', 'edccd1d0813f3171edd3d29c81afb4c25

In [472]:
a['text_piece'].to_list()

["ANNEX I CALCULATING CAPITAL REQUIREMENTS FOR POSITION RISK GENERAL PROVISIONS Netting 1. The excess of an institution's long (short) positions over its short (long) positions in the same equity, debt and convertible issues and identical financial futures, options, warrants and covered warrants shall be its net position in each of those different instruments. In calculating the net position the competent authorities shall allow positions in derivative instruments to be treated, as laid down in points 4 to 7, as positions in the underlying (or notional) security or securities. Institutions' holdings of their own debt instruments shall be disregarded in calculating specific risk under point 14. 2. No netting shall be allowed between a convertible and an offsetting position in the instrument underlying it, unless the competent authorities adopt an approach under which the likelihood of a particular convertible's being converted is taken into account or have a capital requirement to cover

In [444]:
a['text_piece'][0]

"ANNEX I CALCULATING CAPITAL REQUIREMENTS FOR POSITION RISK GENERAL PROVISIONS Netting 1. The excess of an institution's long (short) positions over its short (long) positions in the same equity, debt and convertible issues and identical financial futures, options, warrants and covered warrants shall be its net position in each of those different instruments. In calculating the net position the competent authorities shall allow positions in derivative instruments to be treated, as laid down in points 4 to 7, as positions in the underlying (or notional) security or securities. Institutions' holdings of their own debt instruments shall be disregarded in calculating specific risk under point 14. 2. No netting shall be allowed between a convertible and an offsetting position in the instrument underlying it, unless the competent authorities adopt an approach under which the likelihood of a particular convertible's being converted is taken into account or have a capital requirement to cover 